In [36]:
!pip install nltk


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [37]:
import warnings
warnings.filterwarnings("ignore")

In [38]:
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support # to report on precision and recall
import numpy as np
import re

In [39]:
def StopWordsFromFile(filename):
    stop_words = []
    with open(filename) as file:
        for i in csv.reader(file):
            stop_words.append(i[0].lower())
    return stop_words

def StopWordRemoval(text, lexicon):
    text = text
    for i in lexicon:
        while i in text:
            text.remove(i)
    return text

In [40]:
def FamousPeopleFromFile(filename):
    famous_people=[]
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        for i in reader:
            famous_people.append(i[0].lower())
    return famous_people

def FamousPeopleTransform(text,lexicon):
    famous_matches = []
    text = text
    for i in lexicon:
        if i in text:
            text = re.sub(f"(^|\W){i}", f" |{len(famous_matches)} ", text)
            famous_matches.append(i)
    return [text, famous_matches]

In [41]:
def CaseFold(text):
    return text.lower()

In [42]:
def nGram(text,n=[1]):
    n = [1] + n
    text = text
    ngrams = []

    n.sort()
    for i in range(len(n)):
        start = []
        end = []
        if n[i] == 1:
            continue
        for v in range(n[i]-n[i-1]):
            start.append("<s>")
            end.append("</s>")
        # start = ["<s>" for v in range(i-1-count)]
        # end = ["</s>" for v in range(i-1-count)]
        text = start + text + end
        for j in range(len(text)-n[i]+1):
            ngrams.append(" ".join(text[j:j+n[i]]))
    return text+ngrams  

In [43]:
def NegationWordsFromFile(filename):
    negation_words = []
    with open(filename) as file:
        for i in csv.reader(file):
            negation_words.append(i[0].lower())
    return negation_words

def Negate(text, lexicon):
    text = text
    negation_words = lexicon
    for i in negation_words:
        if i in text:
            matches=[]
            for k in range(len(text)):
                if text[k] == i:
                    matches.append(k)
            for m in matches:
                for j in range(m+1, len(text)):
                    if (text[j]=="not" or "|" in text[j] ):
                        continue
                    text[j] = "not_" + text[j]
                    last = text[j][-1]
                    if (text[j][:8]=="not_not_"):
                        text[j] = text[j][8:]
                    if (text[j][-1]=="." or text[j][-1]=="!" or text[j][-1]=="?"):
                        break
        if text[-1] == "not_" or text[-1] == "":
            text.pop()
    return text

In [44]:
stop_words = StopWordsFromFile("stop_words.csv")
famous_people = FamousPeopleFromFile("famous_people.csv")
negation_words = NegationWordsFromFile("negation.csv")

In [45]:
def load_data(path):
    """Load data from a tab-separated file and append it to raw_data."""
    with open(path,encoding="utf-8") as f:
        reader = csv.reader(f, delimiter='\t')
        for line in reader:
            if line[0] == "Id":  # skip header
                continue
            (label, text) = parse_data_line(line)
            raw_data.append((text, label))

def split_and_preprocess_data(percentage,Case_Folding=True,Punctuation=True,Stop_Word_Removal=False,Famous_People=False,nGramList=[1],Negation_Words=False,
                              Average_Word_Length=False,Types_Per_Tokens=False,Sentiment=False, Mutual_Information="all"):
    """Split the data between train_data and test_data according to the percentage
    and performs the preprocessing."""
    num_samples = len(raw_data)
    num_training_samples = int((percentage * num_samples))
    for (text, label) in raw_data[:num_training_samples]:
        train_data.append((to_feature_vector(pre_process(text,Case_Folding=Case_Folding, Punctuation=Punctuation, Stop_Word_Removal=Stop_Word_Removal,Famous_People=Famous_People, nGramList=nGramList,
                                                         Negation_Words=Negation_Words),
                                             Average_Word_Length=Average_Word_Length,Types_Per_Tokens=Types_Per_Tokens,Sentiment=Sentiment,
                                             Mutual_Information=Mutual_Information),label))
    for (text, label) in raw_data[num_training_samples:]:
        test_data.append((to_feature_vector(pre_process(text,Case_Folding=Case_Folding, Punctuation=Punctuation, Stop_Word_Removal=Stop_Word_Removal,Famous_People=Famous_People, nGramList=nGramList,
                                                        Negation_Words=Negation_Words),
                                             Average_Word_Length=Average_Word_Length,Types_Per_Tokens=Types_Per_Tokens,Sentiment=Sentiment,
                                             Mutual_Information=Mutual_Information),label))

In [46]:
def convert_label(label):
    """Converts the multiple classes into two,
    making it a binary distinction between fake news and real."""
    #return label
    # Converting the multiclass labels to binary label
    labels_map = {
        'true': 'REAL',
        'mostly-true': 'REAL',
        'half-true': 'REAL',
        'false': 'FAKE',
        'barely-true': 'FAKE',
        'pants-fire': 'FAKE'
    }
    return labels_map[label]


def parse_data_line(data_line):
    # Should return a tuple of the label as just FAKE or REAL and the statement
    # e.g. (label, statement)
    return (convert_label(data_line[1]),data_line[2])

In [47]:
# Input: a string of one statement
def pre_process(text,Case_Folding=False,Punctuation=False, Stop_Word_Removal=False,Famous_People=False,nGramList=[1],Negation_Words=False):
    ## Using famous_people, stop_words, negation_words defined above
    text = text
    # Should return a list of tokens
    # DESCRIBE YOUR METHOD IN WORDS
    ## Case Folding
    if Case_Folding==True:
        text = CaseFold(text)
    ## Famous People Matching
    if Famous_People==True:
        text,famous_matches = FamousPeopleTransform(text,famous_people)
    ## RegExp Pattern Match
    if Punctuation ==True:
        pattern = "[^a-zA-Z0-9?!\'\.\|]+"
    else:
        pattern="\W"
    text = re.split(pattern, text)
    ## Stop Words
    if Stop_Word_Removal == True:
        text = StopWordRemoval(text, stop_words)
    ## Lemmatization
    ## I am using the NLTK Lemmatization as Morphological analysis is very complex
    ## Negation Words
    if Negation_Words == True:
        text = Negate(text, negation_words)
    ## Keep Punctuation
    text = " ".join(text)
    replace_pattern = "([\.\?\!])"
    for match in re.finditer(replace_pattern, text):
        text = text.replace(match.group(), f" {match.group()} ")
    text = text.split(" ")
    ## Famous Again
    if Famous_People==True and famous_matches != None:
        for i in range(len(text)):
            if len(text[i])>1 and text[i][0] =="|":
                text[i] = famous_matches[int(text[i][1])]
    ##
    while "" in text:
        text.remove("")
    while "not_" in text:
        text.remove("not_")   
    ## n-Gram
    text = nGram(text, nGramList)
    return text

In [48]:
def AverageWordLength(tokens):
    smn = 0
    for i in tokens:
        smn+= len(i)
    return smn/len(tokens)

In [49]:
def Unique_Word_Per_Total_Words(tokens):
    unique = len(set(tokens))
    total_words = len(tokens)
    return unique/total_words

In [50]:
def SentimentLexiconFromFile(filename):
    positive_sentiment = []
    negative_sentiment = []
    with open(filename) as file:
        for i in csv.reader(file):
            if i[0] == "positive":
                positive_sentiment.append(i[1].lower())
            elif i[0] == "negative":
                negative_sentiment.append(i[1].lower())
    return [negative_sentiment, positive_sentiment]

def getSentiment(tokens, negative_lexicon,positive_lexicon):
    sentiment = {"Positive Sentiment":0, "Negative Sentiment":0}
    for token in tokens:
        if token in negative_lexicon:
            sentiment["Negative Sentiment"]+=1
        elif token in positive_lexicon:
            sentiment["Positive Sentiment"]+=1
    return sentiment

In [51]:
negative_sentiment,positive_sentiment = SentimentLexiconFromFile("sentiment_lexicon.csv")

In [52]:
## Do it again but now with feature selection

def MutualInformationFeatures(filename):
    features = []
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        for i in reader:
            features.append(i[0])
    return features

MI_FAKE = MutualInformationFeatures("Q5_pointwise_mutual_information_FAKE.csv")
MI_REAL = MutualInformationFeatures("Q5_pointwise_mutual_information_REAL.csv")
MI_Features = list(set(MI_FAKE + MI_REAL))
print(len(MI_FAKE))
print(len(MI_REAL))
print(len(MI_Features))

6236
8878
13962


In [53]:
global_feature_dict = {} # A global dictionary of features

def to_feature_vector(tokens, Average_Word_Length=False, Types_Per_Tokens=False,Sentiment=False,Mutual_Information="all"):
    # Should return a dictionary containing features as keys, and weights as values
    # DESCRIBE YOUR METHOD IN WORDS
    
    features_dict = {}
    ## Average word length
    if Average_Word_Length == True:
        features_dict["Average Word Length"] = AverageWordLength(tokens)
    ## Unique_Words/Total_Words
    if Types_Per_Tokens==True:
        features_dict["Unique_Words/Total_Words"] = Unique_Word_Per_Total_Words(tokens)
    ## Sentiment Lexicon
    if Sentiment == True:
        features_dict.update(getSentiment(tokens, negative_sentiment,positive_sentiment))
    ## Mutual_Information Maker
    if Mutual_Information != "all":
        MI_data = MI_Features[:Mutual_Information]
    for token in tokens:
        if Mutual_Information != "all":
            
            if not(token in MI_data):
                continue
        features_dict.setdefault(token, 0)
        global_feature_dict.setdefault(token,0)
        features_dict[token] += 1
        global_feature_dict[token] += 1
    return features_dict

In [54]:
# TRAINING AND VALIDATING OUR CLASSIFIER


def train_classifier(data,SVC_HP={}):
    pipeline =  Pipeline([('svc', LinearSVC(**SVC_HP))])
    return SklearnClassifier(pipeline).train(data)

In [55]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predict_labels(samples, classifier):
    """Assuming preprocessed samples, return their predicted labels from the classifier model."""
    return classifier.classify_many(samples)

def predict_label_from_raw(sample, classifier):
    """Assuming raw text, return its predicted label from the classifier model."""
    return classifier.classify(to_feature_vector(preProcess(reviewSample)))

In [56]:
#solution
from sklearn.metrics import classification_report


def cross_validate(dataset, folds,return_results=False,print_results=True, pointwiseMI=False,SVC_HP={}):
    results = []
    ## 
    fp = []
    fp_dict = {}
    fn = []
    fn_dict = {}
    tp = []
    tp_dict = {}
    tn = []
    tn_dict = {}
    
    if pointwiseMI ==True:
        smoothing_factor = 0
        entropy_c = 0
        features_with_count = {}
        features_real = {}
        features_fake = {}
        information_gain = {}
        mutual_information = {}
    ## 
    ##
    cv_results = {
               "FAKE":{"precision":0, "recall":0, "f1-score":0},
               "REAL":{"precision":0, "recall":0, "f1-score":0},
               "accuracy":0,
               "macro avg":{"precision":0, "recall":0, "f1-score":0},
               "weighted avg":{"precision":0, "recall":0, "f1-score":0}
              }
    
    results_headers = list(cv_results.keys())
    results_sub_headers = list(cv_results["FAKE"].keys())
    ##
    
    fold_size = int(len(dataset)/folds) + 1 
    for i in range(0,len(dataset),int(fold_size)):
        # insert code here that trains and tests on the 10 folds of data in the dataset
        if print_results ==True:
            print("Fold start on items %d - %d" % (i, i+fold_size))
        # FILL IN THE METHOD HERE
        ##
        test = dataset[i:i+fold_size]
        test_x = []
        test_y = []
        for data in test:
            test_x.append(data[0])
            test_y.append(data[1])
        
        train = dataset[0:i]
        train.extend(dataset[i+fold_size:])
        model = train_classifier(train,SVC_HP)
        predicted = predict_labels(test_x, model)
        report = classification_report(test_y, predicted, output_dict=True)
        
        ##
        ##
        
        if i == 0:         
            if pointwiseMI == True:
                ## Counts
                real_prob = test_y.count("REAL")/len(test_y)
                fake_prob = test_y.count("FAKE")/len(test_y)

                for value in range(len(test_y)):
                    for k in list(test_x[value].keys()):
                        features_with_count.setdefault(k,smoothing_factor)
                        features_real.setdefault(k,smoothing_factor)
                        features_fake.setdefault(k,smoothing_factor)
                        features_with_count[k] += 1
                        if test_y[value] == "REAL":
                            features_real[k] += 1
                        elif test_y[value] == "FAKE":
                            features_fake[k] += 1

                feature_count_classes = sum(features_with_count.values())
                feature_count_real = sum(features_real.values())
                feature_count_fake = sum(features_fake.values())

                ##
                ## Mutual Information

                # These dictionaries contain the mutual information for features that occur in that class. This means that I don't have infinities.
                # These infinities arise when when we do MLE and the probability in Pointwise mutual information in the numerator in the log is 0.
                # This means that feature hasn't occured in that class and thus it is a good discriminator of between the classes. So these features are the best.
                # But using the mutual information results directly with infinities isn't that useful. It doesn't tell me the best features for that particular class.
                # So I will print to mr, mf, and mutual_information.
                # Most likely, the best features to keep are the ones that only occur in REAL or FAKE but not both.

                mr = {}
                mf = {}
                mr_removed = {}
                mf_removed = {}

                for feature in list(features_with_count.keys()):
                    if features_real[feature]>0:
                        mr[feature] = np.log2((features_real[feature]/feature_count_real)/((features_with_count[feature]/feature_count_classes)*(real_prob)))
                    if features_fake[feature]>0:
                        mf[feature] = np.log2((features_fake[feature]/feature_count_fake)/((features_with_count[feature]/feature_count_classes)*(fake_prob)))
                    mutual_information.setdefault(feature,0)
                    mutual_information[feature] = fake_prob*(mf[feature] if feature in mf else -np.inf) + real_prob*(mr[feature] if feature in mr else -np.inf)

                with open("Q5_pointwise_mutual_information_FAKE.csv", "w") as file:
                    header = ["Feature", "Mutual Information"]
                    writer = csv.DictWriter(file, fieldnames=header)
                    writer.writeheader()
                    mf_sorted = {k: v for k, v in sorted(mf.items(), key=lambda item: -item[1])}
                    for feature,ig in mf_sorted.items():
                        writer.writerow({header[0]:feature, header[1]: ig})

                with open("Q5_pointwise_mutual_information_REAL.csv", "w") as file:
                    header = ["Feature", "Pointwise Mutual Information"]
                    writer = csv.DictWriter(file, fieldnames=header)
                    writer.writeheader()
                    mr_sorted = {k: v for k, v in sorted(mr.items(), key=lambda item: -item[1])}
                    for feature,ig in mr_sorted.items():
                        writer.writerow({header[0]:feature, header[1]: ig})

                with open("Q5_average_pointwise_mutual_information.csv", "w") as file:
                    header = ["Feature", "Average Pointwise Mutual Information"]
                    writer = csv.DictWriter(file, fieldnames=header)
                    writer.writeheader()
                    mutual_information_sorted = {k: v for k, v in sorted(mutual_information.items(), key=lambda item: -item[1])}
                    for feature,ig in mutual_information_sorted.items():
                        writer.writerow({header[0]:feature, header[1]: ig})
    
        
        ##
        ## Appending the results for all
        results.append(report)
        
        for header in results_headers:
            if header == "accuracy":
                cv_results[header] += report["accuracy"]/folds
                continue
            for sub in results_sub_headers:
                cv_results[header][sub] += report[header][sub]/folds
    if print_results==True:
        for header in results_headers:
            if header == "accuracy":
                print(f"## {header} ## {cv_results[header]}")
                continue
            print(f"## {header} ##")
            for sub in results_sub_headers:
                print(f"{sub} : {cv_results[header][sub]}")
        ##
    return cv_results if return_results==True else None

In [57]:
# MAIN

# loading reviews
# initialize global lists that will be appended to by the methods below
raw_data = []          # the filtered data from the dataset file
train_data = []        # the pre-processed training data as a percentage of the total dataset
test_data = []         # the pre-processed test data as a percentage of the total dataset


# references to the data files
data_file_path = 'fake_news.tsv'

# Do the actual stuff (i.e. call the functions we've made)
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(raw_data), len(train_data), len(test_data)),
      "Preparing the dataset...",sep='\n')

load_data(data_file_path) 

# We split the raw dataset into a set of training data and a set of test data (80/20)
# You do the cross validation on the 80% (training data)
# We print the number of training samples and the number of features before the split
print("Now %d rawData, %d trainData, %d testData" % (len(raw_data), len(train_data), len(test_data)),
      "Preparing training and test data...",sep='\n')

split_and_preprocess_data(0.8, Case_Folding=True,Punctuation=True,Stop_Word_Removal=True,Famous_People=True,nGramList=[1,3],Negation_Words=True,
                              Average_Word_Length=True,Types_Per_Tokens=True,Sentiment=True, Mutual_Information="all")

# We print the number of training samples and the number of features after the split
print("After split, %d rawData, %d trainData, %d testData" % (len(raw_data), len(train_data), len(test_data)),
      "Training Samples: ", len(train_data), "Features: ", len(global_feature_dict), sep='\n')


Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 10241 rawData, 0 trainData, 0 testData
Preparing training and test data...
After split, 10241 rawData, 8192 trainData, 2049 testData
Training Samples: 
8192
Features: 
127099


In [58]:
## Above I have used all the HyperParameters That I made. I will use this to create
## a Mutual Information csv file that contains the Mutual Information between a feature and a
## class. I haven't used the average Pointwise Mutual Information as this involves infinities
## and depending on the amount of smoothing, the values are different.

In [59]:
# will work and output overall performance of p, r, f-score when cv implemented
cross_validate(train_data,10,return_results=False,print_results=True, pointwiseMI=True)

Fold start on items 0 - 820
Fold start on items 820 - 1640
Fold start on items 1640 - 2460
Fold start on items 2460 - 3280
Fold start on items 3280 - 4100
Fold start on items 4100 - 4920
Fold start on items 4920 - 5740
Fold start on items 5740 - 6560
Fold start on items 6560 - 7380
Fold start on items 7380 - 8200
## FAKE ##
precision : 0.5335963244004209
recall : 0.4868556052289856
f1-score : 0.508753186095522
## REAL ##
precision : 0.6296829241390114
recall : 0.6720805139501188
f1-score : 0.6499191279163467
## accuracy ## 0.5915811606391926
## macro avg ##
precision : 0.5816396242697162
recall : 0.5794680595895522
f1-score : 0.5793361570059344
## weighted avg ##
precision : 0.5885712651304381
recall : 0.5915811606391926
f1-score : 0.5888853432144717


In [27]:
## From This result above, we see that the accuracy is much higher than without preprocessing.
## From this, I will now perform feature extraction based on the pointwise mutual information.

In [62]:
## Analysis
## Stop word Removal, Famous_People, Casefold, nGram, Negation_Words

## AVG Word Length, Unique_words_per_total_words, Sentiment, MutualInformation

HyperParameters = {"Case_Folding":[False,True],"Punctuation":[True,False],"Stop_Word_Removal":[True, False], "Famous_People":[True, False], "nGramList":[[1],[2],[3],[5],[1,2,3,4,5],[2,3,5]],
              "Negation_Words":[True, False],
              "Average_Word_Length":[True,False], "Types_Per_Tokens":[True,False], "Sentiment":[True,False],"Mutual_Information":["all",1000,4000],
              "SVC_HP":[{"loss":"hinge"},{"C":0.2},{"C":0.4},{"C":0.8},{"C":1.6}]}

In [63]:
##  there is a lot of Hyperparameters, I will look at them individually and then go from there.

# results_data= [{"hyperparameters":{},score:}]
results_data = []

raw_data = []          # the filtered data from the dataset file
train_data = []        # the pre-processed training data as a percentage of the total dataset
test_data = []         # the pre-processed test data as a percentage of the total dataset

for k in list(HyperParameters.keys()):
    print(len(results_data))
    for v in HyperParameters[k]:
        len(results_data)
        raw_data = []
        train_data = []
        test_data = []

        data_file_path = 'fake_news.tsv'
        load_data(data_file_path) 
        if k =="SVC_HP":
            split_and_preprocess_data(0.8)
        else:
            split_and_preprocess_data(0.8,**{k:v})
        results_data.append({"hyperparameters":{k:v},"score":cross_validate(train_data, 10, return_results=True,print_results=False,pointwiseMI=False,SVC_HP=v if k=="SVC_HP" else {})})

0
2
4
6
8
14
16
18
20
22
25


In [64]:
with open("Q5_Hyperparameter_Analysis.csv","w") as file:
    header = ["hyperparameter", "FAKE precision","FAKE recall","FAKE f1-score",
              "REAL precision", "REAL recall", "REAL f1-score",
              "accuracy","macro avg precision", "macro avg recall", "macro avg f1-score",
              "weighted avg precision", "weighted avg recall", "weighted avg f1-score"]
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    for i in results_data:
        t = {"hyperparameter":i["hyperparameters"], 
             "FAKE precision":i["score"]["FAKE"]["precision"],
             "FAKE recall":i["score"]["FAKE"]["recall"],
             "FAKE f1-score":i["score"]["FAKE"]["f1-score"],
             "REAL precision":i["score"]["REAL"]["precision"],
             "REAL recall":i["score"]["REAL"]["recall"],
             "REAL f1-score":i["score"]["REAL"]["f1-score"],
             "accuracy":i["score"]["accuracy"],
             "macro avg precision":i["score"]["macro avg"]["precision"],
             "macro avg recall":i["score"]["macro avg"]["recall"],
             "macro avg f1-score":i["score"]["macro avg"]["f1-score"],
             "weighted avg precision":i["score"]["weighted avg"]["precision"],
             "weighted avg recall":i["score"]["weighted avg"]["recall"],
             "weighted avg f1-score":i["score"]["weighted avg"]["f1-score"]
            }
        writer.writerow(t)

In [65]:
## Although I would like to combine the above features in hyperparameter turning, this would simply be too long
## (performance) so I have looked at each individually and I will remove features based on that and combine the
## most useful features at the end.

In [68]:
## Based on a simple analysis of Q5_Hyperparameter_Analysis.csv, I have 
## selected the following model:


In [67]:
Features = {"Case_Folding":True, "Punctuation":False,"Stop_Word_Removal":False,
            "nGramList":[5], "Negation_Words":False, "Average_Word_Length":True,
            "Types_Per_Tokens":True,"Sentiment":True,"Mutual_Information":"all"}
HP = {"loss":"hinge","C":0.2}

raw_data = []
train_data = []
test_data = []
    
data_file_path = 'fake_news.tsv'

load_data(data_file_path) 
split_and_preprocess_data(0.8, **Features)
cross_validate(train_data, 10, print_results=True, SVC_HP=HP)

Fold start on items 0 - 820
Fold start on items 820 - 1640
Fold start on items 1640 - 2460
Fold start on items 2460 - 3280
Fold start on items 3280 - 4100
Fold start on items 4100 - 4920
Fold start on items 4920 - 5740
Fold start on items 5740 - 6560
Fold start on items 6560 - 7380
Fold start on items 7380 - 8200
## FAKE ##
precision : 0.553219165942476
recall : 0.5035936221934556
f1-score : 0.5269828714848982
## REAL ##
precision : 0.6423294918195631
recall : 0.686782402636217
f1-score : 0.6636268819633555
## accuracy ## 0.6072149465337018
## macro avg ##
precision : 0.5977743288810194
recall : 0.5951880124148362
f1-score : 0.5953048767241269
## weighted avg ##
precision : 0.6041838747884072
recall : 0.6072149465337018
f1-score : 0.6045518816619581
